In [1]:
import numpy as np, os, sys, pandas as pd, csv, copy
from pathlib import Path

root = '../data'
data_info = np.array(pd.read_table(root +'/list_eval_partition.txt', header=1, delim_whitespace=True))[:,:]
#  np.array(pd.read_table(root +'/train_data/label.txt', delim_whitespace=True))[:,:]

df = pd.read_table(root +'/list_eval_partition.txt', header=1, delim_whitespace=True)

In [2]:
df.head()

,image_name,item_id,evaluation_status
0,img/WOMEN/Dresses/id_00000002/02_1_front.jpg,id_00000002,train
1,img/WOMEN/Dresses/id_00000002/02_2_side.jpg,id_00000002,train
2,img/WOMEN/Dresses/id_00000002/02_4_full.jpg,id_00000002,train
3,img/WOMEN/Dresses/id_00000002/02_7_additional.jpg,id_00000002,train
4,img/WOMEN/Skirts/id_00000003/02_1_front.jpg,id_00000003,train


In [3]:
def get_files_from_path(pathstring):
    """retrives file names from the folder and returns a pandas dataframe with
    four columns: path, filesize, lat, long

    Arguments:
        pathstring {string} -- relative location of file

    Returns:
        [pandas dataframe] -- sorted by the filesize
    """

    filenames = []
    for file in Path(pathstring).glob("**/*.jpg"):
        
        if file.parts[-2] == "gallery" or file.parts[-2] == "query":
            evaluation_status = file.parts[-2]
        else:
            evaluation_status = "train"

        filenames.append((str(file), file.parts[2], evaluation_status))
    
    files_df = pd.DataFrame(list(filenames),
                            columns=["image_name", "item_id", "evaluation_status"])
    
    sorted_files = files_df.sort_values("item_id")
    result_df = sorted_files.reset_index(drop=True)
    return result_df

In [4]:
# data = get_files_from_path("data")

In [5]:
# train, query, gallery = data_info[data_info[:,2]=='train'][:,:2], data_info[data_info[:,2]=='query'][:,:2], data_info[data_info[:,2]=='gallery'][:,:2]


In [6]:
# lab_conv = {x:i for i,x in enumerate(np.unique(np.array([int(x.split('_')[-1]) for x in train[:,1]])))}

# train[:,1] = np.array([lab_conv[int(x.split('_')[-1])] for x in train[:,1]])

In [7]:
train = np.array(pd.read_table(root +'/train_data/label.txt', delimiter=','))[:,:]

In [8]:

train_ys, train_im_paths = [], []
for img_path, key in train:
    train_im_paths.append(os.path.join(root, 'Img', img_path))
    train_ys += [int(key)]


In [9]:
from dataset.huawei import Huawei_Dataset
from dataset.utils import make_transform
from dataset import sampler
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

import torch, math, time, argparse, os
import random, dataset, utils, losses, models
import numpy as np

'/media/stevel/files/computerVision/proxy_anchor_DIGIX/code'

In [10]:
model = 'resnet18'
os.chdir('../data/')
data_root = os.getcwd()
trn_dataset = Huawei_Dataset(
            root = data_root,
            mode = 'train',
            transform = make_transform(
                is_train = True, 
                is_inception = (model == 'bn_inception')
            ))

In [11]:
IPC = 3
sz_batch = 150
nb_workers = 4

if IPC:
    balanced_sampler = sampler.BalancedSampler(trn_dataset, batch_size=sz_batch, images_per_class = IPC)
    batch_sampler = BatchSampler(balanced_sampler, batch_size = sz_batch, drop_last = True)
    dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        num_workers = nb_workers,
        pin_memory = True,
        batch_sampler = batch_sampler
    )
    print('Balanced Sampling')
    
else:
    dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        batch_size = sz_batch,
        shuffle = True,
        num_workers = nb_workers,
        drop_last = True,
        pin_memory = True
    )
    print('Random Sampling')

Balanced Sampling


In [12]:
# for i, data in enumerate(dl_tr):
#     print(i, data)

In [13]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets


In [14]:
balanced_sampler = sampler.BalancedSampler(trn_dataset, batch_size=sz_batch, images_per_class = IPC)
batch_sampler = BatchSampler(balanced_sampler, batch_size = sz_batch, drop_last = True)

dataloader = DataLoader(trn_dataset, batch_size=4,
                        shuffle=True, num_workers=0,
                        batch_sampler = batch_sampler)

                        

ValueError: batch_sampler option is mutually exclusive with batch_size, shuffle, sampler, and drop_last

In [ ]:
# for i, data in enumerate(dataloader):
#     print(i, data)
!pwd

In [16]:
data_transform = transforms.Compose([
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

hymenoptera_dataset = datasets.ImageFolder(root='train_data/',
                                           transform=data_transform)
# dataset_loader = torch.utils.data.DataLoader(hymenoptera_dataset,
#                                              batch_size=4, shuffle=True,
#                                              num_workers=4)

In [17]:
dataset_loader = DataLoader(trn_dataset,
                        batch_size=4, shuffle=True,
                        num_workers=4)

In [21]:
pbar = tqdm(enumerate(dataset_loader))

for batch_idx, (x, y) in pbar:         
     pbar.set_description(
            'test')

3094